In [71]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re


In [22]:

club = "computer"
campus = "Columbus"

d = requests.get(
    #f'https://activities.osu.edu/involvement/student_organizations/find_a_student_org?v=list&s={club}&c={campus}'
    'https://activities.osu.edu/involvement/student_organizations/find_a_student_org?v=list&l=ALL&c=Columbus')
data = d.text
# print(data)


In [223]:
def date_parse(input_str):
    days = ["Sundays", "Mondays", "Tuesdays", "Wednesdays", "Thursdays", "Fridays", "Saturdays"]
    regex = "(?:^|\s)((?:(?:M|Mo|T|Tu|Tue|W|We|Wed|Wedn|Wedne|T|Th|Thu|Thur|F|Fr|S|Sa|Sat|Satu|S|Su)|(?:Mon|Tues|Wednes|Thurs|Fri|Satur|Sun)(?:d|da|day)?)s?)\.?,?\s*?(?:@|at)?\s?(\d{3,4}|(?:\d{1,2}:?(?:\d{2})?))\s?(pm|am)?"
    result = re.search(regex, input_str, re.I)

    if result == None:
        parsed_day = 3
        parsed_time = 700
    else:
        day = result.group(1)
        time = result.group(2)
        meridian = result.group(3)
        print(day, time, meridian)

        # So, if there is no day list the default will be 4
        # parsed_day = 4
        for i in range(len(days)):
            if days[i].__contains__(day):
                parsed_day = i
                print("test")
                break

        # 7 = 0700
        # 7:20 = 0720
        # 720 = 0720
        # 12 = 1200
        # 7am = 0700
        # 7pm = 1900
        # 7:20am = 0720
        # 7:20pm = 1920

        # Default time is 700
        # parsed_time = 700
        if meridian == None:
            if not time.__contains__(":"):
                if len(time) < 3:
                    parsed_time = int(time) * 100
                else:
                    parsed_time = int(time)
            else:
                time = time.replace(":", "")
                parsed_time = int(time)
        else:   # There is a meridian
            if meridian == "am":
                time = time.replace(":", "")
                if len(time) < 3:
                    parsed_time = int(time) * 100
                else:
                    parsed_time = int(time)
            else:
                time = time.replace(":", "")
                if len(time) < 3:
                    # 700
                    parsed_time = int(time) * 100 + 1200
                else:
                    #UNTESTED
                    parsed_time = int(time) + 1200
            
            if time[0:2] == "12": # and meridian == "am":
                parsed_time -= 1200
            # elif time[0:2] == "12" and meridian == "pm":
            #     parsed_time -= 1200
    
    return parsed_day, parsed_time

In [225]:
# Testing if the date parser works correctly
print(date_parse("Mon 12:50pm")) #Tuesday @ 12:00pm Columbus, OH 43210"))
#print(date_parse("Monday @ 7Columbus, OH 43210"))

Mon 12:50 pm
test
(1, 1250)


In [227]:

soup = BeautifulSoup(data, 'html.parser')

# Good tutorial:
#https://medium.com/geekculture/web-scraping-tables-in-python-using-beautiful-soup-8bbc31c5803e

# Finding the holy table
table = soup.find('table', class_='c-table')
# print(table)
# Collecting Ddata
df = pd.DataFrame(columns=["name", "day", "time", "statement", "topics"])
# df = pd.DataFrame()

# Counter is for testing -- takes a long time to make url requests
counter = 0
for row in table.find_all('tr')[1:]:
    # print(f"Our row is {row}")
    # Find all data for each column
    columns = row.find_all('td')
    club_name = columns[0].text.strip()
    
    #-----------------------------EXTRA DATA--------------------------
    if counter < 9999: #50:
        url = columns[0].find(href=True)['href']
        #url = columns[0].find('a').attrs['href'] <-- alternative code
        #print("Found the URL:", url)

        d2 = requests.get(f"https://activities.osu.edu/{url}")
        current_data = d2.text
        club_site = BeautifulSoup(current_data, 'html.parser')
        #print(club_site)

        current_table = club_site.find('table', class_='c-table')
        #print(current_table)

        # current_columns = current_table.find_all('tr')
        # current_campus = current_columns[0].text.strip()
        # print(current_campus)

        current_counter = 0
        for current_row in current_table.find_all('tr')[1:]:
            property = current_row.th.text.strip()

            # Have to do this because the number of properties is not constant
            if property == "Meeting Time and Place:":
                current_time = current_row.find_all('td')[0].text.strip()
                print(current_time)
                current_day, current_time = date_parse(current_time)
                print("TEST")

            # print(property)
            # if current_counter == 14:
            #     current_columns = current_row.find_all('td')
            #     current_campus = current_columns[0].text.strip()
            #     print(current_campus)
            # current_counter += 1
    #-----------------------------------------------------------------

    statement = columns[1].text.strip()
    topics = columns[2].text.strip().split(",")
    if topics[-1] == '':
        topics.pop()
    datarow = pd.DataFrame([{"name":club_name, "day":current_day, "time":current_time, "statement":statement, "topics":set(topics)}])
    df = df.append(datarow, ignore_index=True)
    counter += 1
df

Sundays at 17002121 Tuttle Park PlaceColumbus, OH 43210
Sundays 1700 None
test
TEST
Monday @ 7pmColumbus, OH 43210
Monday 7 pm
test
TEST


C:\Users\lando\AppData\Local\Temp\ipykernel_34800\3465631864.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datarow, ignore_index=True)
C:\Users\lando\AppData\Local\Temp\ipykernel_34800\3465631864.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datarow, ignore_index=True)


Wednesdays @ 7pm (bi-weekly)Enarson Classroom Building2009 Millikin RdColumbus, OH 43210
Wednesdays 7 pm
test
TEST
Columbus, OH 43210
TEST


C:\Users\lando\AppData\Local\Temp\ipykernel_34800\3465631864.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datarow, ignore_index=True)
C:\Users\lando\AppData\Local\Temp\ipykernel_34800\3465631864.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datarow, ignore_index=True)


Sunday 6pmColumbus, OH 43210
Sunday 6 pm
test
TEST
Mondays and Thursdays 9-11pmColumbus, OH 43210
Thursdays 9 None
test
TEST


...

Meetings announced by email, contact leadership.333 W 10th AveColumbus, OH 43210
W 10 None
test
TEST
Various events throughout the yearContact lin.3315@osu.eduColumbus, OH 43210
TEST


C:\Users\lando\AppData\Local\Temp\ipykernel_34800\3465631864.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datarow, ignore_index=True)
C:\Users\lando\AppData\Local\Temp\ipykernel_34800\3465631864.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datarow, ignore_index=True)


Columbus, OH 43210
TEST
Mondays and fridays 8:30-10:30pm1813 n high st.Columbus, OH 43210
fridays 8:30 None


C:\Users\lando\AppData\Local\Temp\ipykernel_34800\3465631864.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datarow, ignore_index=True)


UnboundLocalError: local variable 'parsed_day' referenced before assignment